In [91]:
import pandas as pd
import numpy as np

MIN_N_RATINGS = 1000
DATA_COLLECTION_N = 60
RECOMMENDATION_N = 60
#  training users (75%) are ids 1 to 103869 and testing users (25%) are ids 103870 or greater
FIRST_TESTING_USERID = 103870



In [92]:
# load data with 27k movies and 20M ratings
Movies = pd.read_csv("C:/Users/easht/Documents/CS 673/Movie Database/movie.csv")
Ratings = pd.read_csv("C:/Users/easht/Documents/CS 673/Movie Database/rating.csv")

In [93]:
# merge data to one row per movie/rating
MovieRatings = Movies.merge(Ratings)


In [94]:
# roll-up ratings by movie to determine each movies, average rating and numebr of ratings
MovieRatingsSummary = MovieRatings.groupby(['movieId','title']).agg({'rating': ['count','mean']})
MovieRatingsSummary = MovieRatingsSummary.reset_index()
MovieRatingsSummary.columns = MovieRatingsSummary.columns.map(''.join)
MovieRatingsSummary = MovieRatingsSummary.rename(columns={'ratingcount':'ratingCount', 'ratingmean':'ratingMean' })


In [95]:
# limit possible recommendations to movies to about 1000 movies with >= 5000 ratings each
MoviesGtMinRatings = MovieRatingsSummary.loc[MovieRatingsSummary['ratingCount'] >= MIN_N_RATINGS]
MoviesGtMinRatings = MoviesGtMinRatings.sort_values(by = ['ratingCount'], ascending= False)


In [96]:
# Take out the 30 most-rated movies that will be used for data collection and therefore are 
# not avaialbe to recommend
DataCollectionMovies = MoviesGtMinRatings[:DATA_COLLECTION_N]
DataCollectionMovies

,movieId,title,ratingCount,ratingMean
293,296,Pulp Fiction (1994),67310,4.174231
352,356,Forrest Gump (1994),66172,4.029000
315,318,"Shawshank Redemption, The (1994)",63366,4.446990
587,593,"Silence of the Lambs, The (1991)",63299,4.177057
476,480,Jurassic Park (1993),59715,3.664741
257,260,Star Wars: Episode IV - A New Hope (1977),54502,4.190672
108,110,Braveheart (1995),53769,4.042534
583,589,Terminator 2: Judgment Day (1991),52244,3.931954
2486,2571,"Matrix, The (1999)",51334,4.187186
523,527,Schindler's List (1993),50054,4.310175


In [97]:
#Remove the data collection moves from the movies with sufficent ratings to get the possible rec list
PossibleRecMovies = MoviesGtMinRatings[DATA_COLLECTION_N + 1:]

#Pick the 30 highest-rating movies to become the control model's recommended movies for everyone
PossibleRecMovies = PossibleRecMovies.sort_values(by= 'ratingMean', ascending = False)
ControlRecMovies = PossibleRecMovies[:RECOMMENDATION_N]
controlRecMovieIds = sorted(ControlRecMovies['movieId'].tolist())
ControlRecMovies

,movieId,title,ratingCount,ratingMean
1195,1221,"Godfather: Part II, The (1974)",27398,4.275641
1935,2019,Seven Samurai (Shichinin no samurai) (1954),11611,4.274180
887,904,Rear Window (1954),17449,4.271334
7356,7502,Band of Brothers (2001),4305,4.263182
895,912,Casablanca (1942),24349,4.258327
905,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),6525,4.256935
1169,1193,One Flew Over the Cuckoo's Nest (1975),29932,4.248079
737,750,Dr. Strangelove or: How I Learned to Stop Worr...,23220,4.247287
1186,1212,"Third Man, The (1949)",6565,4.246002
5917,6016,City of God (Cidade de Deus) (2002),12937,4.235410


In [98]:
#limit the users to those in the testing pool only
RatingsTestingUsers = Ratings.loc[Ratings['userId'] >= FIRST_TESTING_USERID]
#isolte the ratings for movies by the testing pool that the control model recommended
RatingsRecMovies = RatingsTestingUsers.loc[RatingsTestingUsers['movieId'].isin(controlRecMovieIds)]
RatingsRecMovies.describe()

,userId,movieId,rating
count,151879.000000,151879.000000,151879.000000
mean,121174.198533,6041.688502,4.201496
std,9955.819177,15720.953961,0.820377
min,103871.000000,745.000000,0.500000
25%,112614.000000,1131.000000,4.000000
50%,121136.000000,1221.000000,4.000000
75%,129839.000000,3030.000000,5.000000
max,138493.000000,79132.000000,5.000000


In [99]:
#group the ratings by user and re-flatten the column names
UserPerformacneSummary = RatingsRecMovies.groupby(['userId']).agg({'rating': ['count','mean']})
UserPerformacneSummary = UserPerformacneSummary.reset_index()
UserPerformacneSummary.columns = UserPerformacneSummary.columns.map(''.join)
UserPerformacneSummary.describe()

,userId,ratingcount,ratingmean
count,22738.000000,22738.000000,22738.000000
mean,121172.917759,6.679523,4.189327
std,9956.893839,7.270040,0.651442
min,103871.000000,1.000000,0.500000
25%,112623.250000,2.000000,4.000000
50%,121131.500000,4.000000,4.250000
75%,129789.750000,9.000000,4.600000
max,138493.000000,60.000000,5.000000


In [100]:
#summarize results
total_users = Ratings['userId'].max()
total_testing_users = total_users - FIRST_TESTING_USERID + 1
users_with_performance_scores = UserPerformacneSummary['userId'].count()
possible_rec_movie_n = PossibleRecMovies['title'].count()
n_recs_measurable = RatingsRecMovies['rating'].count()
avg_rec_rating = RatingsRecMovies['rating'].mean()
q1_rec_rating = RatingsRecMovies['rating'].quantile(0.25)
q3_rec_rating = RatingsRecMovies['rating'].quantile(0.75)
avg_all_rating = Ratings['rating'].mean()
q1_all_rating = Ratings['rating'].quantile(0.25)
q3_all_rating = Ratings['rating'].quantile(0.75)
users_with_measured_recs = UserPerformacneSummary['userId'].count()
mean_recs_per_user = UserPerformacneSummary['ratingcount'].mean()
q1_recs_per_user = UserPerformacneSummary['ratingcount'].quantile(0.25)
q3_recs_per_user = UserPerformacneSummary['ratingcount'].quantile(0.75)
print('Out of ' + str(total_users) + ' total users ' + str(total_testing_users) + ' were selected to test the model.')
print('Each user was recommended ' + str(RECOMMENDATION_N) + ' movies. Only movies with at least ' + str(MIN_N_RATINGS) +
    ' ratings were eligible to be recommended.')
print('The ' + str(DATA_COLLECTION_N) + ' movies that were rated most often are also not \
eligible since it is assumed we will use those for data collection.')
print('Out of the remaining ' + str(possible_rec_movie_n) + ' movies, the control model recommends the \
30 movies with the higest average rating to each user.')
print('Out of the ' + str(total_users * RECOMMENDATION_N) + ' recs made, ' + str(n_recs_measurable) + ' have ratings we can use\
 to test the control model\'s performance.')
print('The control model average rec rating is {:.2f} out of 5.00 with most falling between {:.2f} and {:.2f}'.format(avg_rec_rating, q1_rec_rating, q3_rec_rating ))
print('This is higher than the original data set where the average rating is {:.2f} with most falling between {:.2f} and {:.2f}'.format(avg_all_rating, q1_all_rating, q3_all_rating))


Out of 138493 total users 34624 were selected to test the model.
Each user was recommended 60 movies. Only movies with at least 1000 ratings were eligible to be recommended.
The 60 movies that were rated most often are also not eligible since it is assumed we will use those for data collection.
Out of the remaining 3098 movies, the control model recommends the 30 movies with the higest average rating to each user.
Out of the 8309580 recs made, 151879 have ratings we can use to test the control model's performance.
The control model average rec rating is 4.20 with most falling between 4.00 and 5.00
This is higher than the original data set where the average rating is 3.53 with most falling between 3.00 and 4.00
